In [1]:
magma_dir = '/home/ubuntu/magma/'
bucket_dir = '/home/ubuntu/s3/'
transformers_dir = '/home/ubuntu/transformers/'
cache_dir = bucket_dir+'.cache/'

## **Fine-tuning**

In [2]:
finetune_script = '"'+transformers_dir+'examples/seq2seq/finetune_trainer.py"'
eval_script = '"'+transformers_dir+'examples/seq2seq/run_eval.py"'

### **Config**

In [3]:
import sys
sys.path.insert(0, magma_dir)
import config

import torch
torch.manual_seed = config.SEED

import wandb
wandb.login()

project_name = 'hp_search_para_wordembed'
%env WANDB_PROJECT=$project_name

Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: Currently logged in as: marcoabrate (use `wandb login --relogin` to force relogin)


env: WANDB_PROJECT=hp_search_para_wordembed


### HP search

In [4]:
model_name_or_path = 'sshleifer/distilbart-cnn-12-6'

In [5]:
data_dir = '"'+bucket_dir+'datasets/karger_books_para_wordembed/bart/st/"'

output_dir = '"'+bucket_dir+'fine-tuning/hyperparameters_test"'

log_dir = output_dir + '/logs'

In [6]:
from transformers import AutoConfig
model_config = AutoConfig.from_pretrained(model_name_or_path, use_cache=False)
model_config.min_length = config.ONE_BULLET_MIN_LEN
model_config.max_length = config.ONE_BULLET_MAX_LEN
model_config.length_penalty = config.LENGTH_PENALTY
model_config.no_repeat_ngram_size = config.NO_REPEAT_NGRAM_SIZE

model_config.task_specific_params['summarization']['min_length'] = config.ONE_BULLET_MIN_LEN
model_config.task_specific_params['summarization']['max_length'] = config.ONE_BULLET_MAX_LEN
model_config.task_specific_params['summarization']['length_penalty'] = config.LENGTH_PENALTY
model_config.task_specific_params['summarization']['no_repeat_ngram_size'] = config.NO_REPEAT_NGRAM_SIZE
model_config_dir = '"'+bucket_dir+'fine-tuning/'+\
    model_name_or_path.replace('/', '?')+'_config"'
model_config.save_pretrained(model_config_dir[1:-1])

In [7]:
!python3 $finetune_script \
--model_name_or_path $model_name_or_path \
--config_name $model_name_or_path \
--tokenizer_name $model_name_or_path \
--cache_dir $cache_dir \
--data_dir $data_dir \
--hyperparameters_search \
--fp16 \
--freeze_embeds --freeze_encoder \
--sortish_sampler \
--task summarization \
--max_source_length 512 \
--max_target_length 60 \
--val_max_target_length 60 \
--num_train_epochs 1 \
--n_train 50 \
--n_val 5 \
--logging_steps 1 --logging_first_step \
--per_device_train_batch_size 1 --per_device_eval_batch_size 1 \
--evaluation_strategy steps --eval_steps 3 \
--predict_with_generate \
--output_dir $output_dir \
--overwrite_output_dir \
--seed $config.SEED \
--run_name $output_dir

100%|████████████████████████████████████████| 306M/306M [00:03<00:00, 96.2MB/s]
02/01/2021 15:18:21 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: True
02/01/2021 15:18:21 - INFO - __main__ -   Training/evaluation parameters Seq2SeqTrainingArguments(output_dir='/home/ubuntu/s3/fine-tuning/hyperparameters_test', overwrite_output_dir=True, do_train=False, do_eval=True, do_predict=False, model_parallel=False, evaluation_strategy=<EvaluationStrategy.STEPS: 'steps'>, prediction_loss_only=False, per_device_train_batch_size=1, per_device_eval_batch_size=1, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=1.0, max_steps=-1, warmup_steps=0, logging_dir='runs/Feb01_15-18-21_ip-172-31-39-35', logging_first_step=True, logging_step

[INFO|modeling_utils.py:1140] 2021-02-01 15:18:56,420 >> All model checkpoint weights were used when initializing BartForConditionalGeneration.

[INFO|modeling_utils.py:1149] 2021-02-01 15:18:56,420 >> All the weights of BartForConditionalGeneration were initialized from the model checkpoint at sshleifer/distilbart-cnn-12-6.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BartForConditionalGeneration for predictions without further training.
02/01/2021 15:18:56 - INFO - utils -   using task specific params for summarization: {'early_stopping': True, 'length_penalty': 2.0, 'max_length': 142, 'min_length': 56, 'no_repeat_ngram_size': 3, 'num_beams': 4}
[INFO|modeling_utils.py:1024] 2021-02-01 15:18:56,971 >> loading weights file https://huggingface.co/sshleifer/distilbart-cnn-12-6/resolve/main/pytorch_model.bin from cache at /home/ubuntu/s3/.cache/b336fa0b874ea92e3e22f07a7e6f8fa9da01221759c33abeb2679d6d98fe7755.585965cf7e82e4536033cd21d

Batches: 100%|██████████| 1/1 [00:00<00:00, 129.77it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 138.71it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 138.75it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 139.22it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 138.65it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 137.17it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 139.10it/s]
                                             
100%|██████████| 5/5 [00:03<00:00,  1.60it/s]
                                             [INFO|trainer.py:862] 2021-02-01 15:19:58,765 >> 

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 6/6 [00:11<00:00,  1.99s/it]

{'epoch': 0.96}
[I 2021-02-01 15:19:58,766] Trial 0 finished with value: 168.60604810714722 and parameters: {'learning_rate': 0.0006885533250495481, 'gradient_accumulation_steps': 8}. Best is trial 0 with value: 168.60604810714722.
[INFO|trainer.py:557] 2021-02-01 15:19:58,766

                                              {'loss': 2.6997389793395996, 'learning_rate': 6.180590747731665e-05, 'epoch': 0.24}
 25%|██▌       | 3/12 [00:00<00:02,  3.65it/s][INFO|trainer.py:1412] 2021-02-01 15:20:27,917 >> ***** Running Evaluation *****
[INFO|trainer.py:1413] 2021-02-01 15:20:27,917 >>   Num examples = 5
[INFO|trainer.py:1414] 2021-02-01 15:20:27,917 >>   Batch size = 1

100%|██████████| 5/5 [00:02<00:00,  1.60it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 111.74it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 117.12it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 134.52it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 136.92it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 139.26it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 140.66it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 139.05it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 139.19it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 122.62it/s]


Batches: 100%|██████████| 1/1 [00:00<

[INFO|modeling_utils.py:1024] 2021-02-01 15:20:45,689 >> loading weights file https://huggingface.co/sshleifer/distilbart-cnn-12-6/resolve/main/pytorch_model.bin from cache at /home/ubuntu/s3/.cache/b336fa0b874ea92e3e22f07a7e6f8fa9da01221759c33abeb2679d6d98fe7755.585965cf7e82e4536033cd21d76c486af3d6b1c2a34b3a847840d4e7fe9d8844
[INFO|modeling_utils.py:1140] 2021-02-01 15:21:04,764 >> All model checkpoint weights were used when initializing BartForConditionalGeneration.

[INFO|modeling_utils.py:1149] 2021-02-01 15:21:04,764 >> All the weights of BartForConditionalGeneration were initialized from the model checkpoint at sshleifer/distilbart-cnn-12-6.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BartForConditionalGeneration for predictions without further training.
02/01/2021 15:21:04 - INFO - utils -   using task specific params for summarization: {'early_stopping': True, 'length_penalty': 2.0, 'max_length': 142, 'min_length': 56, 'no

...

wandb:                                                                                
wandb: Find user logs for this run at: /home/ubuntu/magma/fine-tuning/wandb/run-20210201_152220-2v95q0vm/logs/debug.log
wandb: Find internal logs for this run at: /home/ubuntu/magma/fine-tuning/wandb/run-20210201_152220-2v95q0vm/logs/debug-internal.log
wandb: Run summary:
wandb:                           train/loss 3.12997
wandb:                  train/learning_rate 0.0
wandb:                          train/epoch 0.96
wandb:                                _step 12
wandb:                             _runtime 39
wandb:                           _timestamp 1612192982
wandb:                            eval/loss 1.736
wandb:                eval/rouge1_precision 36.73
wandb:                   eval/rouge1_recall 59.78
wandb:                 eval/rouge1_fmeasure 44.26
wandb:                eval/rouge2_precision 22.04
wandb:                   eval/rouge2_recall 37.53
wandb:                 eval/rouge2_fmeasur

wandb:                                                                                
wandb: Find user logs for this run at: /home/ubuntu/magma/fine-tuning/wandb/run-20210201_152302-2qz0av63/logs/debug.log
wandb: Find internal logs for this run at: /home/ubuntu/magma/fine-tuning/wandb/run-20210201_152302-2qz0av63/logs/debug-internal.log
wandb: Run summary:
wandb:                           train/loss 48.24683
wandb:                  train/learning_rate 0.00155
wandb:                          train/epoch 0.48
wandb:                                _step 3
wandb:                             _runtime 26
wandb:                           _timestamp 1612193011
wandb:                            eval/loss 15.5178
wandb:                eval/rouge1_precision 0.8
wandb:                   eval/rouge1_recall 0.59
wandb:                 eval/rouge1_fmeasure 0.68
wandb:                eval/rouge2_precision 0.0
wandb:                   eval/rouge2_recall 0.0
wandb:                 eval/rouge2_fmeasure 


wandb: Waiting for W&B process to finish, PID 8146
wandb: Program ended successfully.
wandb:                                                                                
wandb: Find user logs for this run at: /home/ubuntu/magma/fine-tuning/wandb/run-20210201_152331-3imp95dn/logs/debug.log
wandb: Find internal logs for this run at: /home/ubuntu/magma/fine-tuning/wandb/run-20210201_152331-3imp95dn/logs/debug-internal.log
wandb: Run summary:
wandb:                           train/loss 13.20456
wandb:                  train/learning_rate 0.00042
wandb:                          train/epoch 0.24
wandb:                                _step 3
wandb:                             _runtime 26
wandb:                           _timestamp 1612193040
wandb:                            eval/loss 6.81755
wandb:                eval/rouge1_precision 6.32
wandb:                   eval/rouge1_recall 14.12
wandb:                 eval/rouge1_fmeasure 8.47
wandb:                eval/rouge2_precision 0.0
wan

 60%|██████    | 3/5 [00:01<00:00,  2.06it/s]


 80%|████████  | 4/5 [00:02<00:00,  1.68it/s]


100%|██████████| 5/5 [00:02<00:00,  1.60it/s]



Batches: 100%|██████████| 1/1 [00:00<00:00, 118.88it/s]




Batches: 100%|██████████| 1/1 [00:00<00:00, 119.68it/s]




Batches: 100%|██████████| 1/1 [00:00<00:00, 133.30it/s]




Batches: 100%|██████████| 1/1 [00:00<00:00, 134.08it/s]




Batches: 100%|██████████| 1/1 [00:00<00:00, 133.86it/s]




Batches: 100%|██████████| 1/1 [00:00<00:00, 136.40it/s]




Batches: 100%|██████████| 1/1 [00:00<00:00, 132.64it/s]




Batches: 100%|██████████| 1/1 [00:00<00:00, 134.12it/s]




Batches: 100%|██████████| 1/1 [00:00<00:00, 136.98it/s]




Batches: 100%|██████████| 1/1 [00:00<00:00, 137.68it/s]{'eval_loss': 1.6786811351776123, 'eval_rouge1_precision': 32.54, 'eval_rouge1_recall': 52.33, 'eval_rouge1_fmeasure': 38.96, 'eval_rouge2_precision': 16.869999999999997, 'eval_rouge2_recall': 31.64, 'eval_rouge2_fmeasure': 20.9, 'eval_rougeL_precision': 26.91

[W 2021-02-01 15:24:42,103] Trial 8 failed because of the following error: RuntimeError('CUDA out of memory. Tried to allocate 16.00 MiB (GPU 0; 14.76 GiB total capacity; 13.66 GiB already allocated; 13.75 MiB free; 13.83 GiB reserved in total by PyTorch)',)
Traceback (most recent call last):
  File "/home/ubuntu/miniconda3/envs/magma/lib/python3.6/site-packages/optuna/_optimize.py", line 211, in _run_trial
    value_or_values = func(trial)
  File "/home/ubuntu/transformers/src/transformers/integrations.py", line 168, in _objective
    trainer.train(model_path=model_path, trial=trial)
  File "/home/ubuntu/transformers/src/transformers/trainer.py", line 622, in train
    self.model = model.to(self.args.device)
  File "/home/ubuntu/miniconda3/envs/magma/lib/python3.6/site-packages/torch/nn/modules/module.py", line 612, in to
    return self._apply(convert)
  File "/home/ubuntu/miniconda3/envs/magma/lib/python3.6/site-packages/torch/nn/modules/module.py", line 359, in _apply
    module._a